In [1]:
import ee 
import geemap 
import xarray as xr 
import numpy as np 
import matplotlib.pyplot as plt 
import geopandas as gpd

/home/lutz/miniconda3/envs/urban_programming/lib/python3.9/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
#wd setting
import os
wd = '/home/lutz/master/code/second_sem/urban/presentation/code'
os.chdir(wd)
print('Current working directory:', os.getcwd())

Current working directory: /home/lutz/master/code/second_sem/urban/presentation/code


In [3]:
ee.Initialize(project="ee-lutz-training") #Authenticate with your Google account !!Insert your project name here!! 
ee.Authenticate() 

True

## Loading the AOI's

In [4]:
munich=gpd.read_file("../data/aoi_munich.geojson").to_crs("EPSG:4326")
cologne=gpd.read_file("../data/aoi_cologne.geojson").to_crs("EPSG:4326")

#extract geometry
munich_geom=munich.geometry.iloc[0]
cologne_geom=cologne.geometry.iloc[0]

#convert geometry info to GeoJSON dict
munich_geojson=munich_geom.__geo_interface__
cologne_geojson=cologne_geom.__geo_interface__

#create EE Geometry
munich_ee=ee.Geometry(munich_geojson)
cologne_ee=ee.Geometry(cologne_geojson)
#plot map using geemap
Map = geemap.Map()  # zoomed out to show both cities
Map.add_gdf(munich, layer_name="Munich AOI")
Map.add_gdf(cologne, layer_name="Cologne AOI")
Map.add_basemap("OpenStreetMap")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [9]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


munich_props=(
  ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
  .filterDate("2024-06-01","2024-08-31")
  .filterBounds(munich_ee)
  .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 15))
  .map(mask_s2_clouds)
  .map(lambda img: img.clip(munich_ee))
  .map(lambda img: img.select(["B8", "B4", "B3", "B2"]).rename(["Nir","Red", "Green", "Blue"]))
  .map(lambda img: img.addBands(img.normalizedDifference(['Nir', 'Red']).rename('NDVI')))  

) 
footprint = munich_props.geometry()
Map.addLayer(footprint, {'color': 'red'}, 'Image footprints')

cologne_props=(
  ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
  .filterDate("2024-06-01","2024-08-31")
  .filterBounds(cologne_ee)
  .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 15))
  .map(mask_s2_clouds)
  .map(lambda img: img.clip(cologne_ee))
  .map(lambda img: img.select(["B8", "B4", "B3", "B2"]).rename(["Nir","Red", "Green", "Blue"]))
  #ndvi calc
  .map(lambda img: img.addBands(img.normalizedDifference(['Nir', 'Red']).rename('NDVI')))
)

munich_median = munich_props.median()
cologne_median = cologne_props.median()

#plotting the images
Map = geemap.Map()
Map.centerObject(munich_ee, 11)  # Center on Munich AOI

# Add median true-color composite
Map.addLayer(munich_median, {'bands': ['Red', 'Green', 'Blue'], 'min': 0, 'max': 0.2}, 'Munich Median RGB')
Map.addLayer(munich_ee, {}, 'Munich AOI')
Map.addLayer(munich_geom, {}, 'Munich AOI')
ndvi_viz = {
    'min': 0.0,
    'max': 0.8,
    'palette': ['blue', 'white', 'green'],
}
Map.addLayer(munich_median.select('NDVI'), ndvi_viz, 'Munich NDVI')
Map

Map(center=[48.153162064787196, 11.547401603636363], controls=(WidgetControl(options=['position', 'transparent…